In [1]:
# Import the libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

import keras
from keras.models import Sequential
from keras.layers import Dense

import matplotlib.pyplot as plt

In [2]:
# Download data and split training & test sets 
concrete_data = pd.read_csv('https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [3]:
# Split columns into Predictors and Target
concrete_data_columns = concrete_data.columns

P = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
T = concrete_data['Strength'] # Strength column

In [4]:
# number of predictors
n_cols = P.shape[1]

In [5]:
# Define the regression model with one hidden layer
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(1))
   
    # compile the model
    model.compile(optimizer='adam', loss='mean_squared_error')
    
    return model

# Build the model
model = regression_model()

In [6]:
# Wecreate a 50-cycle training & evaluation loop. We would train the model on a 50-epoch fit -per cycle- and evaluate using MSE per cycle. We record each cycle MSE into a list.

score_mse_list = [] # Create a list to record the MSEs per cycle  
cycle  = 0 # Initialise the cycles
for looping in range(50): # Loop 50 times
    # Split the Predictors & Target data, holding 30% for testing
    P_train, P_test, T_train, T_test = train_test_split(P, T,
                                                    test_size = 0.3,
                                                    random_state = 1)
    
    #Initialise the cycle MSE
    score_mse = 0
    
    # Fit the model 
    model.fit(P_train, T_train, epochs=50, verbose=2)
    
    # Generate predicted test target values
    T_test_hat = model.predict(P_test)
    
    # Cycle evaluation using scikit-learn mean_squared_error
    score_mse = mean_squared_error(T_test, T_test_hat) 
    
    # Append the loop MSE on the list
    score_mse_list.append(score_mse)

Epoch 1/50
23/23 - 0s - loss: 16446.3574 - 380ms/epoch - 17ms/step
Epoch 2/50
23/23 - 0s - loss: 6648.6860 - 17ms/epoch - 739us/step
Epoch 3/50
23/23 - 0s - loss: 4558.5010 - 17ms/epoch - 721us/step
Epoch 4/50
23/23 - 0s - loss: 3569.6731 - 17ms/epoch - 745us/step
Epoch 5/50
23/23 - 0s - loss: 2818.1262 - 17ms/epoch - 739us/step
Epoch 6/50
23/23 - 0s - loss: 2168.6104 - 17ms/epoch - 742us/step
Epoch 7/50
23/23 - 0s - loss: 1619.4996 - 16ms/epoch - 696us/step
Epoch 8/50
23/23 - 0s - loss: 1137.9100 - 16ms/epoch - 696us/step
Epoch 9/50
23/23 - 0s - loss: 808.5850 - 16ms/epoch - 698us/step
Epoch 10/50
23/23 - 0s - loss: 586.0825 - 16ms/epoch - 696us/step
Epoch 11/50
23/23 - 0s - loss: 463.8110 - 17ms/epoch - 739us/step
Epoch 12/50
23/23 - 0s - loss: 396.9624 - 16ms/epoch - 696us/step
Epoch 13/50
23/23 - 0s - loss: 355.2773 - 16ms/epoch - 696us/step
Epoch 14/50
23/23 - 0s - loss: 324.1232 - 16ms/epoch - 696us/step
Epoch 15/50
23/23 - 0s - loss: 302.3558 - 16ms/epoch - 692us/step
Epoch 16/5

In [7]:
# Printing the list of MSEs and verify the its length
print(score_mse_list)
len(score_mse_list)

[129.17344433968623, 119.95794303793366, 110.43171446863053, 86.24900631415422, 73.92014825767076, 63.53831322136912, 60.10401852555687, 57.38553251813833, 57.96091962605975, 58.69777177269872, 58.602147865649144, 57.279202302086944, 56.93302388286077, 56.2859217486338, 55.59307728661188, 54.97924230389032, 55.76938141084258, 58.43107308739358, 54.63551025198043, 55.1015175695193, 56.68703908011977, 54.52632000641574, 54.51885189866628, 54.45100477702045, 53.60580040192374, 52.902121401062466, 54.809716120878306, 53.675183860646406, 53.4679772059856, 54.243050703565224, 52.462525566116426, 53.79926503260122, 52.59751826304238, 53.92852762593403, 52.48332949393242, 52.91840064029988, 54.75606452889992, 53.64354774663996, 53.37226228081921, 54.04353827818835, 53.145173216779256, 53.62524727591788, 53.711076176037324, 54.09466761788313, 53.52725203081745, 53.49016661278133, 52.885206230280566, 53.82439747499137, 53.65642940613047, 54.01833042032219]


50

In [8]:
# Calculate and report the mean & std of the MSEs from the list
print('The training values mean of the 50-cycle MSEs is :', np.mean(score_mse_list))
print('The training values standard deviation of the 50-cycle MSEs is :', np.std(score_mse_list))

The training values mean of the 50-cycle MSEs is : 59.87797802332132
The training values standard deviation of the 50-cycle MSEs is : 16.228756561895302
